In [1]:
import polars as pl
import numpy as np
import pandas as pd
from module import network, util, dataset

In [2]:
# %%
import gc
import os
import pickle
import random
import joblib
import pandas as pd
# import polars as pd
from tqdm import tqdm
from sklearn.metrics import average_precision_score as APS
import polars as pl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
import numpy as np
import torch.nn.functional as F
import math
import time
import sys
import datetime
import pytz
import matplotlib.pyplot as plt


from module import network, dataset, util
from importlib import reload

# %%
class Config:
    PREPROCESS = False
    KAGGLE_NOTEBOOK = False
    DEBUG = False
    MODEL = 'lstm'
    SEED = 42
    EPOCHS = 9*2
    BATCH_SIZE = 4096
    LR = 1e-4
    WD = 1e-6
    PATIENCE = 3
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    EARLY_STOPPING = False
    NUM_CV = 1
    VAL_INDEX = [0]
    NOTEBOOK = False
    LOAD_MODEL = True
    # models配下
    MODEL_PATH = "lstm/lstm_fold1_54.pt"
    
    
if Config.DEBUG:
    n_rows = 10**4
else:
    n_rows = None



# %%
if Config.KAGGLE_NOTEBOOK:
    RAW_DIR = "/kaggle/input/leash-BELKA/"
    PROCESSED_DIR = "/kaggle/input/belka-enc-dataset"
    OUTPUT_DIR = ""
    MODEL_DIR = ""
else:
    RAW_DIR = "../data/raw/"
    PROCESSED_DIR = "../data/processed/"
    OUTPUT_DIR = "../data/result/"
    MODEL_DIR = "../models/"
    LOG_DIR = "../data/logs"
    LOSS_DIR = "../data/losses"

TRAIN_DATA_NAME = "train_enc.parquet"


In [3]:

def predict_in_batches(model, data, batch_size):
    model.eval()  # Set model to evaluation mode
    preds = []
    for i in range(0, data.size(0), batch_size):
        batch = data[i:i+batch_size].to(Config.DEVICE)
        with torch.no_grad():
            batch_preds = torch.sigmoid(model(batch))  # apply sigmoid
        preds.append(batch_preds.detach().cpu())
    return torch.cat(preds, dim=0)

In [30]:
model_paths = ["lstm_fold0_72.pt", "lstm_fold1_54.pt", "lstm_fold2_54.pt", "lstm_fold3_54.pt", "lstm_fold4_63.pt", "lstm_fold5_54.pt", "lstm_fold6_54.pt", "lstm_fold7_54.pt", "lstm_fold8_54.pt"]

for i, path in enumerate(model_paths):
    model_path = os.path.join(MODEL_DIR, "lstm")
    model_path = os.path.join(model_path, path)
    model = network.LSTMModel().to(Config.DEVICE)
    # 0以外のモデルはcompileする. 訓練のときのミス
    if i not in [0]:
        model = torch.compile(model)
    model.load_state_dict(torch.load(model_path))
    print(f"model loaded {model_path}")
    
    # テストデータの読み込み
    test = pd.read_parquet(os.path.join(PROCESSED_DIR, "test_enc.parquet"))
    test_tensor = torch.tensor(test.values, dtype=torch.float32).to(Config.DEVICE)
    preds = predict_in_batches(model, test_tensor, Config.BATCH_SIZE)
    preds = preds.numpy()
        
    tst = pl.read_parquet(os.path.join(RAW_DIR, "test.parquet"), n_rows=None).to_pandas()

    # 'binds'列を追加して初期化
    tst['binds'] = 0

    # ブールマスクの作成
    mask_BRD4 = (tst['protein_name'] == 'BRD4').values
    mask_HSA = (tst['protein_name'] == 'HSA').values
    mask_sEH = (tst['protein_name'] == 'sEH').values

    # 各マスクに対応する予測値を代入
    tst.loc[mask_BRD4, 'binds'] = preds[mask_BRD4][:, 0]
    tst.loc[mask_HSA, 'binds'] = preds[mask_HSA][:, 1]
    tst.loc[mask_sEH, 'binds'] = preds[mask_sEH][:, 2]



    submission = tst[['id', 'binds']].copy()
    # 'id'と'binds'列をCSVに出力
    file_name = path.replace(".pt", ".csv")
    file_name = os.path.join(OUTPUT_DIR, file_name)
    submission.to_csv(file_name, index=False)
    print(f"file saved {file_name}")

        
        

model loaded ../models/lstm/lstm_fold0_72.pt


/tmp/ipykernel_3779917/2601361681.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[6.88809081e-08 3.92980269e-07 1.07668195e-04 ... 3.66526801e-05
 8.99846691e-06 5.53948157e-05]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tst.loc[mask_BRD4, 'binds'] = preds[mask_BRD4][:, 0]


file saved ../data/result/lstm_fold0_72.csv
model loaded ../models/lstm/lstm_fold1_54.pt


/tmp/ipykernel_3779917/2601361681.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.7605805e-04 7.9816079e-04 7.5823354e-04 ... 2.2782087e-05 3.0156509e-06
 9.8169599e-05]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tst.loc[mask_BRD4, 'binds'] = preds[mask_BRD4][:, 0]


file saved ../data/result/lstm_fold1_54.csv
model loaded ../models/lstm/lstm_fold2_54.pt


...

file saved ../data/result/lstm_fold6_54.csv
model loaded ../models/lstm/lstm_fold7_54.pt


/tmp/ipykernel_3779917/2601361681.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.3680089e-03 1.0163675e-03 3.3375507e-03 ... 5.9616697e-05 3.2398468e-05
 5.3227897e-05]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tst.loc[mask_BRD4, 'binds'] = preds[mask_BRD4][:, 0]


file saved ../data/result/lstm_fold7_54.csv
model loaded ../models/lstm/lstm_fold8_54.pt


/tmp/ipykernel_3779917/2601361681.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[4.4682383e-04 4.9676150e-05 5.5513014e-03 ... 1.5417008e-06 1.4295779e-06
 1.8419667e-04]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tst.loc[mask_BRD4, 'binds'] = preds[mask_BRD4][:, 0]


file saved ../data/result/lstm_fold8_54.csv


In [42]:
# ensemble
for path in model_paths:
    file_name = path.replace(".pt", ".csv")
    file_name = os.path.join(OUTPUT_DIR, file_name)
    submission = pd.read_csv(file_name)
    if path == model_paths[0]:
        ensemble = submission
    else:
        ensemble['binds'] += submission['binds']

ensemble['binds'] /= len(model_paths)
ensemble.to_csv(os.path.join(OUTPUT_DIR, "lstm_ensemble.csv"), index=False)

In [46]:
lstm = pd.read_csv(os.path.join(OUTPUT_DIR, "lstm_ensemble_0.393.csv"))
cnn = pd.read_csv(os.path.join(OUTPUT_DIR, "cnn_0.416.csv"))

ensemble = cnn * 0.9 + lstm * 0.1
ensemble.to_csv(os.path.join(OUTPUT_DIR, "lstm_cnn_ensemble.csv"), index=False)

In [49]:
blender = pd.read_csv(os.path.join(OUTPUT_DIR, "blender_0.448.csv"))
ensemble = pd.read_csv(os.path.join(OUTPUT_DIR, "lstm_cnn_ensemble.csv"))

ensemble['binds'] = ensemble['binds'] * 0.15 + blender['binds'] * 0.85
ensemble.to_csv(os.path.join(OUTPUT_DIR, "lstm_cnn_blender_ensemble.csv"), index=False)

In [4]:
model_path = os.path.join(MODEL_DIR, Config.MODEL_PATH)

In [5]:
model = network.LSTMModel().to(Config.DEVICE)
model = torch.compile(model)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [6]:
# テストデータの読み込み
test = pd.read_parquet(os.path.join(PROCESSED_DIR, "test_enc.parquet"))
test_tensor = torch.tensor(test.values, dtype=torch.float32).to(Config.DEVICE)
preds = predict_in_batches(model, test_tensor, Config.BATCH_SIZE)
preds = preds.numpy()

KeyboardInterrupt: 

In [11]:

tst = pl.read_parquet(os.path.join(RAW_DIR, "test.parquet"), n_rows=None).to_pandas()

# 'binds'列を追加して初期化
tst['binds'] = 0

# ブールマスクの作成
mask_BRD4 = (tst['protein_name'] == 'BRD4').values
mask_HSA = (tst['protein_name'] == 'HSA').values
mask_sEH = (tst['protein_name'] == 'sEH').values

# 各マスクに対応する予測値を代入
tst.loc[mask_BRD4, 'binds'] = preds[mask_BRD4][:, 0]
tst.loc[mask_HSA, 'binds'] = preds[mask_HSA][:, 1]
tst.loc[mask_sEH, 'binds'] = preds[mask_sEH][:, 2]



submission = tst[['id', 'binds']].copy()
# 'id'と'binds'列をCSVに出力
submission.to_csv(os.path.join(OUTPUT_DIR,'submission.csv'), index=False)


/tmp/ipykernel_1005998/3638645060.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.7605805e-04 7.9816079e-04 7.5823354e-04 ... 2.2782087e-05 3.0156509e-06
 9.8169599e-05]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tst.loc[mask_BRD4, 'binds'] = preds[mask_BRD4][:, 0]
